# Propagation Models: FSPL, Rain, Gas, Scintillation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jman4162/opensatcom/blob/main/notebooks/04_propagation_models.ipynb)

Compare propagation models and visualize their impact on link performance.

In [ ]:
# Install opensatcom (uncomment for Colab)
# !pip install -q opensatcom

In [ ]:
import numpy as np
from opensatcom.core.models import PropagationConditions
from opensatcom.propagation import (
    FreeSpacePropagation,
    RainAttenuationP618,
    GaseousAbsorptionP676,
    ScintillationLoss,
    CompositePropagation,
)

## 1. Individual Model Behavior

In [ ]:
fspl = FreeSpacePropagation()
rain = RainAttenuationP618(rain_rate_mm_per_hr=25.0, availability_target=0.99)
gas = GaseousAbsorptionP676(water_vapor_density_g_m3=7.5)
scint = ScintillationLoss(availability_target=0.99)

cond = PropagationConditions()
freqs_ghz = np.array([4, 8, 12, 18, 20, 28, 30, 40, 50])
elev = 30.0
range_m = 38_000_000.0

print(f"{'Freq (GHz)':>10} {'FSPL':>10} {'Rain':>10} {'Gas':>10} {'Scint':>10} {'Total':>10}")
print("-" * 65)
for f in freqs_ghz:
    l_fspl = fspl.total_path_loss_db(f * 1e9, elev, range_m, cond)
    l_rain = rain.total_path_loss_db(f * 1e9, elev, range_m, cond)
    l_gas = gas.total_path_loss_db(f * 1e9, elev, range_m, cond)
    l_scint = scint.total_path_loss_db(f * 1e9, elev, range_m, cond)
    total = l_fspl + l_rain + l_gas + l_scint
    print(f"{f:>10.0f} {l_fspl:>10.2f} {l_rain:>10.2f} {l_gas:>10.2f} {l_scint:>10.2f} {total:>10.2f}")

## 2. Rain Attenuation 3D Surface

In [ ]:
from opensatcom.viz.heatmaps import plot_rain_attenuation_surface

freqs = np.linspace(4, 50, 25)
elevs = np.linspace(5, 90, 20)

fig = plot_rain_attenuation_surface(freqs, elevs, rain_rate_mm_per_hr=25.0)
fig.show()

## 3. Comparison: Clear Sky vs Full Atmospheric Model

In [ ]:
import plotly.graph_objects as go

elevations = np.linspace(5, 90, 50)
freq = 20e9

losses_fspl = [fspl.total_path_loss_db(freq, e, range_m, cond) for e in elevations]

composite_full = CompositePropagation([fspl, rain, gas, scint])
losses_full = [composite_full.total_path_loss_db(freq, e, range_m, cond) for e in elevations]

additional = [f - s for f, s in zip(losses_full, losses_fspl)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=elevations, y=losses_fspl, name="FSPL only", line=dict(color="blue")))
fig.add_trace(go.Scatter(x=elevations, y=losses_full, name="FSPL+Rain+Gas+Scint", line=dict(color="red")))
fig.update_layout(
    title=f"Total Path Loss at {freq/1e9:.0f} GHz",
    xaxis_title="Elevation (deg)",
    yaxis_title="Total Loss (dB)",
    template="plotly_white"
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=elevations, y=additional, name="Additional atmospheric loss", line=dict(color="orange", width=3)))
fig.update_layout(
    title=f"Additional Atmospheric Loss at {freq/1e9:.0f} GHz (Rain+Gas+Scint)",
    xaxis_title="Elevation (deg)",
    yaxis_title="Additional Loss (dB)",
    template="plotly_white"
)
fig.show()

---

**Next:** See `05_trade_studies.ipynb` for DOE + Pareto analysis.